In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mtl

Notebook com Funções

Função Calcula Features

Classe Stop Execution

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

Função apagar todos so ficheiros de uma diretoria

In [ ]:
def delete_files_in_directory(directory_path):
   try:
     with os.scandir(directory_path) as entries:
       for entry in entries:
         if entry.is_file():
            os.unlink(entry.path)
     print("All files deleted successfully.")
   except OSError:
     print("Error occurred while deleting files.")

In [ ]:
def calc_feats (df_league, team):
  team_stats = dict()
  df_team = df_league[(df_league['HomeTeam']==team) | (df_league['AwayTeam']==team )]
  df_team_form = df_team.tail(5)

  # calc shots
  team_shots = df_team.apply(lambda x : x['HS']
    if x['HomeTeam'] == team
    else x['AS'], axis=1).sum()

  # calc shots on target
  team_shots_ot = df_team.apply(lambda x : x['HST']
    if x['HomeTeam'] == team
    else x['AST'], axis=1).sum()

  # calc shots against
  team_shots_vs = df_team.apply(lambda x : x['AS']
    if x['HomeTeam'] == team
    else x['HS'], axis=1).sum()

  # calc shots on target against
  team_shots_ot_vs = df_team.apply(lambda x : x['AST']
    if x['HomeTeam'] == team
    else x['HST'], axis=1).sum()

  # calc points
  team_points = df_team.apply(lambda x :
    3
      if ((x['HomeTeam'] == team and x['FTR'] == 'H')
      or (x['AwayTeam'] == team and x['FTR'] == 'A'))
    else
    1
      if ( x['FTR'] == 'D')
    else
    0, axis=1).sum()

  # calc form
  team_form = df_team_form.apply(lambda x :
    3
      if ((x['HomeTeam'] == team and x['FTR'] == 'H')
      or (x['AwayTeam'] == team and x['FTR'] == 'A'))
    else
    1
      if ( x['FTR'] == 'D')
    else
    0, axis=1).sum()

  team_stats["team"] = team
  team_stats["shots"] = team_shots
  team_stats["shots_ot"] = team_shots_ot
  team_stats["shots_vs"] = team_shots_vs
  team_stats["shots_ot_vs"] = team_shots_ot_vs
  team_stats["points"] = team_points
  team_stats["form"] = team_form


  return team_stats

Função de criação de um registo de Liga

In [ ]:
def create_league(league_record, leagues_config_path):

  df_league = pd.DataFrame(league_record)

  if os.path.exists(leagues_config_path) == False:
  # criar ficheiro e adidionar registo da liga ao csv de ligas com header
    df_league.to_csv(leagues_config_path, mode='x', header=True, index=False)
  else:
    # adidionar registo da liga ao csv de ligas
    df_league.to_csv(leagues_config_path, mode='a', header=False, index=False)

Função para obter um registo de liga segundo o seu nome

In [ ]:
def retrieve_league(league,leagues_config_path ):
  df_leagues=pd.read_csv(leagues_config_path)
  df_league = df_leagues.loc[df_leagues['league'] == league]

  return ([] if len(df_league) == 0 else df_league.to_dict('records')[0] )

Função criiação da estrutura de pastas de uma liga

In [ ]:
def make_directory_structure(league):

  main_path_league = MAIN_PATH + 'Data_Football/{0}/'.format(league)
  clean_path_league = main_path_league + 'Clean'
  model_path_league = main_path_league + 'Model'
  predicts_path_league = main_path_league + 'Predicts'
  experiments_path_league = predicts_path_league + '/Experiments'
  raw_path_league = main_path_league + 'Raw'
  stats_path_league = main_path_league + 'Stats'

  if os.path.exists(main_path_league) == False:
    os.mkdir(main_path_league)
  if os.path.exists(clean_path_league) == False:
    os.mkdir(clean_path_league)
  if os.path.exists(model_path_league) == False:
    os.mkdir(model_path_league)
  if os.path.exists(predicts_path_league) == False:
    os.mkdir(predicts_path_league)
  if os.path.exists(experiments_path_league) == False:
    os.mkdir(experiments_path_league)
  if os.path.exists(raw_path_league) == False:
    os.mkdir(raw_path_league)
  if os.path.exists(stats_path_league) == False:
    os.mkdir(stats_path_league)

In [ ]:
def add_numbers(a, b):
    return a + b

função split data - iNCREMENTAL

In [ ]:
def split_data (fixture, number_matches ,df_to_split):
  matches = fixture*number_matches
  df1=df_to_split.iloc[:matches, :]
  df2=df_to_split.iloc[matches:matches + number_matches, :]

  return df1, df2

função split data - Fixture_number

In [ ]:
def split_data_fixture_number(fixture ,df_to_split):
  df1 = df_to_split.loc[df_to_split['GW'] != fixture]
  df2 = df_to_split.loc[df_to_split['GW'] == fixture]

  return df1, df2

Função Calculo de Bet Size

In [ ]:
def calculate_bet_size(uniform_bet_size , fraction_of_total, total_funds_previousweek):

  # Calculo bet_size = fraction_of_total da dezena anterior do total_funds_previousweek, fraction_of_total = 0.05
  # (ex: para total_funds_previousweek 63,3 --> dezena anterior = 60 -->  Bet_Size = 0,05*60 = 3.0 )
  if (total_funds_previousweek < TOTAL_FUNDS_DEFAULT):
    return fraction_of_total*(total_funds_previousweek - (total_funds_previousweek % 10))

  if uniform_bet_size == False :
    # Calculo bet_size = fraction_of_total da dezena anterior do total_funds_previousweek, fraction_of_total = 0.05
    # (ex: para total_funds_previousweek 63,3 --> dezena anterior = 60 -->  Bet_Size = 0,05*60 = 3.0 )
    bet_size = fraction_of_total*(total_funds_previousweek - (total_funds_previousweek % 10))
  else:
    # Bet_size uniform igual a fraction_of_total do capital inicial
    bet_size = fraction_of_total*TOTAL_FUNDS_DEFAULT

  return bet_size



Função de Paragem de processamento

In [ ]:
def exit(): raise StopExecution